# Resumo

Neste notebook:
- Uma rede neural de 3 camadas densas é criada e testada com Cross Validation
- Outra rede neural, também de 3 camadas, mas com Dropout em 2, é criada e testada com Cross Validation
- Os resultados (pontuação média e desvio padrão) de ambas são comparados.

A conclusão é que o uso de Dropout foi efetivo em aumentar a precisão média e o desvio padrão do modelo.<br><br>

O dataset utilizado é o Breast Cancer Winsconsin, do repositório UC Irvine.<br>
Possui 569 amostras, cada uma com 32 variáveis (1 ID, 1 target e 30 atributos).<br>
Target: 0 para begnino, 1 para maligno (classificação binária).<br>
Não há dados faltantes.<br>

# Importação das bibliotecas, classes e funções

In [4]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
from scikeras.wrappers import KerasClassifier #wrapper de classificador
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k


# Importação dos dados

In [5]:
X = pd.read_csv('Deep Learning com Python de A a Z/Parte 1 - Redes Neurais Artificiais/classificação binária/entradas_breast.csv')
y = pd.read_csv('Deep Learning com Python de A a Z/Parte 1 - Redes Neurais Artificiais/classificação binária/saidas_breast.csv')

In [6]:
def create_network():
    k.clear_session() #limpa a sessão do backend do Tensorflow
    network = Sequential([
        tf.keras.layers.InputLayer(shape=(30,)),
        tf.keras.layers.Dense(units = 16, activation='relu', kernel_initializer='random_uniform'),
        tf.keras.layers.Dense(units = 16, activation='relu', kernel_initializer='random_uniform'),
        tf.keras.layers.Dense(units = 1, activation='sigmoid')
    ])
    adam1 = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
    network.compile(optimizer=adam1, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return network

In [7]:
network = KerasClassifier(model = create_network, epochs = 100, batch_size=10)

Na linha acima, um modelo wrapped (traduzido) para Scikit-Learn foi retornado.<br>
Sua estrutura foi retornada pela função create_network, seu treinamento terá 100 épocas e utilizará batches com 10 amostras.

Na linha abaixo, o modelo network será treinado e avaliado por cross-validation utilizando a função cross_val_score.<br>
A base de dados será dividia em 10 partes, ou seja, o modelo será treinado e avaliado 10x.<br>
A métrica utilizada para avaliar a precisão do modelo em cada iteração da CV é a função accuracy, ou seja, uma simples porcentagem de acertos.

In [8]:
results = cross_val_score(estimator = network, X = X, y = y, cv = 10, scoring='accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - binary_accuracy: 0.5321 - loss: 0.9579
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7466 - loss: 0.5615
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8050 - loss: 0.4815
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8093 - loss: 0.5089
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8186 - loss: 0.6505
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8127 - loss: 0.5139
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.8413 - loss: 0.4505
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.7439 - loss: 0.8193
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8410 - loss: 0.4356
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8047 - loss: 0.5068
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8171 -

In [9]:
results

array([0.80701754, 0.75438596, 0.80701754, 0.78947368, 0.68421053,
       0.8245614 , 0.78947368, 0.75438596, 0.70175439, 0.82142857])

A precisão final do modelo será a média dessas 10:

In [10]:
results.mean()

0.7733709273182956

Para medir a consistência do modelo, podemos analisar o desvio padrão das precisões de cada iteração da Cross-Validation:

In [11]:
results.std()

0.046381195409264886

# Aplicando Dropout

In [15]:
def create_network2():
    k.clear_session() #limpa a sessão do TensorFlow
    network = Sequential([
        tf.keras.layers.InputLayer(shape=(30,)),
        tf.keras.layers.Dense(units = 16, activation='relu', kernel_initializer='random_uniform'),
        tf.keras.layers.Dropout(rate=0.2), #aplica um dropout de 20% na primeira camada oculta
        tf.keras.layers.Dense(units = 16, activation='relu', kernel_initializer='random_uniform'),
        tf.keras.layers.Dropout(rate=0.2), #aplcia um dropout de 20% na segunda camada oculta
        tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
    ])
    adam1 = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
    network.compile(optimizer=adam1, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return network


In [16]:
network = KerasClassifier(model=create_network2, batch_size=10, epochs=100)

In [17]:
results2 = cross_val_score(estimator=network, X=X, y=y, cv=10, scoring='accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - binary_accuracy: 0.5463 - loss: 2.0113
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6557 - loss: 0.6834
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6387 - loss: 0.6623
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6483 - loss: 0.7477
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6641 - loss: 0.6498
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6667 - loss: 0.5948
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7524 - loss: 0.5870
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.6566 - loss: 0.6244
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7639 - loss: 0.5476
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7386 - loss: 0.5545
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.7358 -

In [18]:
results2

array([0.85964912, 0.94736842, 0.84210526, 0.87719298, 0.87719298,
       0.9122807 , 0.87719298, 0.92982456, 0.8245614 , 0.92857143])

In [19]:
results

array([0.80701754, 0.75438596, 0.80701754, 0.78947368, 0.68421053,
       0.8245614 , 0.78947368, 0.75438596, 0.70175439, 0.82142857])

In [20]:
results2.mean()

0.887593984962406

In [21]:
results.mean()

0.7733709273182956

In [22]:
results2.std()

0.03846138021906814

In [23]:
results.std()

0.046381195409264886

Ou seja, obtivemos um aumento grande na precisão média do modelo e uma diminuição substancial no desvio padrão das pontuações. Isso mostra que o Dropout resultou em uma rede neural mais precisa e consistente.